In [4]:
from ghostnet.ghost_net import ghost_net
# from keras.datasets import mnist
import pandas as pd
import numpy as np
import torch
from torch.nn.utils.rnn import pack_padded_sequence
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn,optim
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
import SimpleITK as sitk
from sklearn.decomposition import PCA
from keras.preprocessing.image import ImageDataGenerator
import os

# 检验GPU是否可用
print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

True
cuda:0


In [5]:
def dataLoad(path1, path2):
    paths1 = os.listdir(path1)
    paths2 = os.listdir(path2)
    dataset = []
    labelset = []
    
    for file in paths1:
        if file.find('mhd')>=0:
            filepath = os.path.join(path1, file)
            data = sitk.GetArrayFromImage(sitk.ReadImage(filepath))
            data = data.reshape((1,256,256))
            dataset.append(data)
            labelset.append(1)
            
    for file in paths2:
        if file.find('mhd')>=0:
            filepath = os.path.join(path2, file)
            data = sitk.GetArrayFromImage(sitk.ReadImage(filepath))
            data = data.reshape((1,256,256))
            dataset.append(data)
            labelset.append(0)
        
    dataset = np.array(dataset)
    labelset = np.array(labelset)
    return dataset, labelset

In [6]:
train_data, train_label = dataLoad('Training/Diseased', 'Training/Healthy')
test_data, test_label = dataLoad('Testing1/Diseased', 'Testing1/Healthy')

print(train_data.shape)
print(test_data.shape)

train_data = train_data.reshape((378,256,256,1))
print(train_data.shape)


(378, 1, 256, 256)
(50, 1, 256, 256)
(378, 256, 256, 1)


In [7]:
train_data = train_data.reshape((378,1,256,256))
test_data = test_data.reshape((-1,1,256,256))
train_data = torch.FloatTensor(train_data)
test_data = torch.FloatTensor(test_data)
train_label = torch.LongTensor(train_label)
test_label = torch.LongTensor(test_label)

In [8]:
test_label.shape

torch.Size([50])

In [9]:
class MyDataset(Dataset):
    
    # 初始化
    def __init__(self, pics, lable):
        # 读入数据
        self.data = pics
        self.lable = lable
        
    # 返回df的长度
    def __len__(self):
        return len(self.data)
    
    # 获取第idx+1列的数据
    def __getitem__(self, idx):
        return self.data[idx],self.lable[idx]

In [10]:
my_dataset_train = MyDataset(train_data, train_label)
my_dataset_test = MyDataset(test_data, test_label)
train_loader = DataLoader(dataset = my_dataset_train, batch_size = 8, shuffle = True)
test_loader = DataLoader(dataset = my_dataset_test, batch_size = 8, shuffle = True)

In [13]:
model = ghost_net(width_mult=1.0,num_classes=2).to(device)
# model = Net().to(device)
# optimizer = optim.Adam(model.parameters(),lr=0.001)  # 优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [19]:
num_epochs = 10
n_total_steps = len(train_loader)
LossList = [] # 记录每一个epoch的loss
AccuryList = [] # 每一个epoch的accury
for epoch in range(num_epochs):
    # -------
    # 开始训练
    # -------
    model.train() # 切换为训练模型
    totalLoss = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device) # 图片大小转换
        labels = labels.to(device)
        # 正向传播以及损失的求取
        outputs = model(images)
#         labels = labels.squeeze(1)
        loss = criterion(outputs, labels)
        totalLoss = totalLoss + loss.item()
        # 反向传播
        optimizer.zero_grad() # 梯度清空
        loss.backward() # 反向传播
        optimizer.step() # 权重更新
        if (i+1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, n_total_steps, totalLoss/(i+1)))
    LossList.append(totalLoss/(i+1))
    # ---------
    # 开始测试
    # ---------
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1) # 预测的结果
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(correct)
        acc = 100.0 * correct / total # 在测试集上总的准确率
        AccuryList.append(acc)
        print('Accuracy of the network on the {} test images: {} %'.format(total, acc))
print("模型训练完成")

Epoch [1/10], Step [10/48], Loss: 0.2948
Epoch [1/10], Step [20/48], Loss: 0.3067
Epoch [1/10], Step [30/48], Loss: 0.2720
Epoch [1/10], Step [40/48], Loss: 0.2631
31
Accuracy of the network on the 50 test images: 62.0 %
Epoch [2/10], Step [10/48], Loss: 0.0915
Epoch [2/10], Step [20/48], Loss: 0.0733
Epoch [2/10], Step [30/48], Loss: 0.1310
Epoch [2/10], Step [40/48], Loss: 0.1356
32
Accuracy of the network on the 50 test images: 64.0 %
Epoch [3/10], Step [10/48], Loss: 0.1418
Epoch [3/10], Step [20/48], Loss: 0.1030
Epoch [3/10], Step [30/48], Loss: 0.1046
Epoch [3/10], Step [40/48], Loss: 0.0935
37
Accuracy of the network on the 50 test images: 74.0 %
Epoch [4/10], Step [10/48], Loss: 0.0307
Epoch [4/10], Step [20/48], Loss: 0.0214
Epoch [4/10], Step [30/48], Loss: 0.0278
Epoch [4/10], Step [40/48], Loss: 0.0240
32
Accuracy of the network on the 50 test images: 64.0 %
Epoch [5/10], Step [10/48], Loss: 0.0085
Epoch [5/10], Step [20/48], Loss: 0.0167
Epoch [5/10], Step [30/48], Loss: 